In [1]:
from ipynb.fs.full.gbs import *

## Stochastic Tests

Tests are not currently comprehensive nor rigorous. Outputs need to be checked by eye.

TESTS ARE ABLE TO RUN != CODE IS WORKING CORRECTLY

### test `zpf`

In [2]:
def check_expectations(arr: np.ndarray):
    # avg z
    exp_val = arr.mean()

    # avg z^T z for every column z in arr
    # if statement optimizes speed
    trans = np.trace((arr @ arr.T if arr.shape[0] < arr.shape[1] else arr.T @ arr)) / arr.shape[1]

    # avg z^H z for every column z in arr
    herm = np.linalg.norm(arr, axis=0).mean()
    
    print(f"E[z] = {exp_val} should be 0.")
    print(f"E[z^T z] = {trans} should be 0.")
    print(f"E[z^H z] = {herm} should be 1.")

In [3]:
vac = zpf(1000000, 2)
check_expectations(vac)

E[z] = (-3.256030296929916e-05+5.5304898914374974e-05j) should be 0.
E[z^T z] = (0.0006073510769163261+0.0004888338996092957j) should be 0.
E[z^H z] = 0.9399747118451505 should be 1.


### test `threshold_detector` and `h_or_v_detector`

In [4]:
a = zpf(100,2)

#print(a)

td = threshold_detector(a, 1)
print(td)

hv = h_or_v_detector(a, 1)
print(hv)

td_any = td.any(axis=0)
assert(np.array_equal(hv, td_any))

[[ True  True  True False False  True False False False False False False
  False False False False False False False False False False False False
  False False False False False False False  True  True False False False
   True False False False False  True False False False  True False False
  False False False False  True  True False False False False  True  True
  False False  True False  True  True False False False False False False
  False False False False  True  True  True  True False False False False
  False  True False False False False False False False False False False
  False False False False]
 [False  True False False False False  True False False False False False
  False False  True False  True False False False False False False False
  False False False False False  True False False False False False False
  False False False False False False False False False False False False
  False  True  True False False False False False False  True False False
  False Fal

### test `ent`, `zpf`, `pbs`, coincidence counters

In [5]:
l,r = ent(1000, 2)
lzpf = zpf(1000, 2)
rzpf = zpf(1000, 2)

In [6]:
lt, lr = pbs(l, lzpf)
rt, rr = pbs(r, rzpf)

dlt, dlr, drt, drr = map(h_or_v_detector, (lt, lr, rt, rr))
detections = np.array([dlt, dlr, drt, drr])

coincidence_counts = get_all_coincidence_counts(detections)
dlt_and_drt = sum([coincidence_counts[event] for event in [(1,0,1,0), (1,0,1,1), (1,1,1,0), (1,1,1,1)]])
assert(get_coincidence_count(detections[[0,2],]) == dlt_and_drt)

print_all_coincidence_counts(detections)


0000 54
0001 1
0010 2
0011 0
0100 0
0101 169
0110 0
0111 5
1000 3
1001 0
1010 180
1011 9
1100 0
1101 13
1110 8
1111 556


## QM Tests

Tests are not currently comprehensive nor rigorous. Outputs need to be checked by eye.

TESTS ARE ABLE TO RUN != CODE IS WORKING CORRECTLY

### test `get_fock_basis_states`

Takes a long time and eats up memory very quickly. The advantage of using `@cache` can be seen by running `get_fock_basis_states` twice in a row, or running it with smaller inputs after it finishes - after the first run, future calls are almost instantaneous.

In [2]:
num_src = 4
n = 2*num_src
m = 4*num_src
get_fock_basis_states(n, m)

array([[0, 0, 0, ..., 0, 0, 8],
       [0, 0, 0, ..., 0, 1, 7],
       [0, 0, 0, ..., 0, 2, 6],
       ...,
       [7, 0, 0, ..., 0, 1, 0],
       [7, 0, 0, ..., 0, 0, 1],
       [8, 0, 0, ..., 0, 0, 0]])

### test `qr_haar` and `HilbertSpaceUnitary`

In [8]:
U = qr_haar(100) #np.array([[0, 1], [1, 0]], dtype=complex)

#print(U)

UH = HilbertSpaceUnitary(U, 1)

zers = np.zeros(100, dtype=int)
zert = np.zeros(100, dtype=int)

zers[0] = 1
zert[0] = 1

print(abs(UH.get_entry(zers, zert))**2)
print(abs(UH[zers, zert])**2)

0.012103304420125418
0.012103304420125418


### test `get_hilbert_space_unitary_matrix`

In [9]:
hadamard = np.array([[1, 1], [1, -1]]) / sqrt(2)

print("Hadamard with 1 photon, 2 modes:")
hsu_had = get_hilbert_space_unitary_matrix(hadamard, 1, p=True) # p=True prints resulting unitary
print()

print("Hadamard with 2 photons, 2 modes:")
hsu_had = get_hilbert_space_unitary_matrix(hadamard, 2, p=True)
print()

m = 6
U = qr_haar(m)

print(f"Haar random unitary with 1 photon, {m} modes:")
hsu_haar = get_hilbert_space_unitary_matrix(U, 1, p=True)
print()

print(f"Haar random unitary with 2 photons, {m} modes:")
hsu_haar = get_hilbert_space_unitary_matrix(U, 2, p=True)
print()

Hadamard with 1 photon, 2 modes:
[1 0]:	[0.71-0.j 0.71-0.j]
[0 1]:	[ 0.71-0.j -0.71+0.j]

Hadamard with 2 photons, 2 modes:
[0 2]:	[ 0.5 +0.j -0.71+0.j  0.5 +0.j]
[1 1]:	[-0.71+0.j  0.  +0.j  0.71+0.j]
[2 0]:	[0.5 +0.j 0.71+0.j 0.5 +0.j]

Haar random unitary with 1 photon, 6 modes:
[1 0 0 0 0 0]:	[-0.07-0.35j  0.67+0.41j -0.08+0.09j  0.1 -0.09j  0.03+0.4j   0.22+0.15j]
[0 1 0 0 0 0]:	[ 0.32+0.25j  0.21-0.08j -0.03-0.03j  0.79+0.15j  0.04+0.07j -0.22-0.29j]
[0 0 1 0 0 0]:	[ 0.26-0.3j   0.25-0.04j  0.61-0.47j -0.19+0.06j -0.23-0.14j -0.09-0.26j]
[0 0 0 1 0 0]:	[-0.09-0.15j -0.01+0.24j  0.12-0.24j  0.19+0.19j  0.65-0.46j -0.06+0.36j]
[0 0 0 0 1 0]:	[-0.3 -0.62j -0.23-0.13j  0.18+0.44j  0.32+0.24j -0.19-0.11j -0.01-0.11j]
[0 0 0 0 0 1]:	[ 0.08+0.19j  0.2 -0.33j  0.27+0.14j  0.04+0.25j -0.28+0.08j -0.14+0.74j]

Haar random unitary with 2 photons, 6 modes:
[0 0 0 0 0 2]:	[-0.53-0.2j  -0.03-0.3j   0.07-0.04j -0.04-0.09j -0.27-0.01j -0.06+0.02j
 -0.04+0.1j  -0.12-0.03j -0.2 +0.26j  0.05+0.08j 